In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
# Setup
!pip install -q torch torchvision bitsandbytes faiss-gpu-cu12
!pip install -q accelerate transformers sentence-transformers
!pip install -q langchain langchain-community
!pip install -q pypdf
!pip install gradio

import torch
import locale
from google.colab import drive

locale.getpreferredencoding = lambda: "UTF-8"

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
!pip install numpy==1.26.4
!pip install accelerate==0.26.0
!pip install transformers==4.41.0
!pip install sentence-transformers==2.4.0
!pip install peft==0.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.5.2
    Uninstalling accelerate-1.5.2:
      Successfully uninstalled accelerate-1.5.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 108.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.4.1
    Uninstalling sentence-transformers-3.4.1:
      Successfully uninstalled sentence-transformers-3.4.1
ERROR: Operation cancelled by user
^C
^C


In [1]:
# Prepare relevant files
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader('/content/drive/MyDrive/My Projects/Files/About Me AI', glob="**/*.md", loader_cls=TextLoader)
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=102)
"""
worse performance:
    chunk_size=512, chunk_overlap=51
"""

files = []
async for file in loader.alazy_load():
    files.append(file)
print("total number of files:", len(files), "\n\n")
# print(f"{files[-1].metadata}\n")
# print(files[-1].page_content)

chunked_docs = splitter.split_documents(files)

total number of files: 12 




In [2]:
# Initialize vector database & retriever
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

vector_db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
"""
worse performance:
    "intfloat/e5-base-v2"
    "BAAI/bge-base-en-v1.5"
    "sentence-transformers/all-mpnet-base-v2"
    "BAAI/bge-m3" # taking too long
"""
retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={"k": 8, "fetch_k": 40, "lambda_mult": 0.5})
"""
worse performance:
    search_type="similarity"

    "lambda_mult": 0.25
    "lambda_mult": 0.75
"""

<ipython-input-2-a4b5325c8ca6>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vector_db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still opt

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'\nworse performance:\n    search_type="similarity"\n\n    "lambda_mult": 0.25\n    "lambda_mult": 0.75\n'

In [ ]:
docs1 = retriever.invoke("What's your name?")
[doc.page_content for doc in docs1]

["My name is Liren Zhang, nicknamed Leo. My student email is [lzhangeg@connect.ust.hk](mailto:lzhangeg@connect.ust.hk). My Hong Kong phone number is \\+852 54334153\\. I am currently pursuing a Bachelor’s Degree at HKUST a.k.a. Hong Kong University of Science and Technology. It is a public research university in Sai Kung District, New Territories, Hong Kong. It is organized into four academic schools focusing on business and management, engineering, humanities and social science, and science. In most major university rankings, it is consistently ranked within Asia's top 15, and is regarded as one of the top young universities in the world.",
 'Both of these activities evoke a sense of fulfillment in me as I am glad to bring joy and amazement to others through my performances. Moreover, they are also opportunities for me to exhibit an unfamiliar yet unique side of myself.',
 '### **Relationships:**\n\nIn relationships, I am loyal, dependable, and practical. My strength lies in my commit

In [ ]:
docs2 = retriever.invoke("Where are you currently studying at?")
[doc.page_content for doc in docs2]

['. \n\n**Liren Zhang** \n\nNew Territories, Hong Kong SAR lirenleonidas@gmail.com (852) 543334153 github.com/leonidasly/Projects \n\n**SUMMARY** \n\nAspiring multilingual undergraduate seeking to leverage practical knowledge in the tech industry. Demonstrates a strong passion for producing data-driven insights and is passionate about working with data. Self-motivated learner who is always open to new concepts. \n\n**EDUCATION** \n\n**Science (Group A)** \n\nThe Hong Kong University of Science and Technology • New Territories, Hong Kong SAR • PRESENT • 3.78 \n\n• Dean’s List. \n\n**US High School Diploma/AP** \n\nNakornpayap International School • Chiang Mai, THAILAND • 2024 • 4.17 \n\n• Received the Valedictorian Award for having the highest cumulative GPA in my grade. \n\n**PROJECTS** \n\n**AI Course Selection Assistant (Prototype)** \n\nHKUST • October 2024 \\- December 2024 \n\n• Built a chatbot with LangChain and Hugging Face.',
 'Besides AI, I also love to play the guitar. In fac

In [ ]:
docs3 = retriever.invoke("Could you tell me about yourself and describe your background in brief?")
[doc.page_content for doc in docs3]

['### **Personality Traits:**\n\nAs an ISTJ, I exhibit a blend of traits: I am introverted, observant, thinking, judging, and assertive. I tend to be logical and fact-oriented, excelling in analytical tasks. However, I may find it difficult to understand or express emotions, leading to potential misunderstandings in personal relationships. My integrity, dependability, and ability to provide stability make me a valued friend, partner, and colleague. \n\n### **Career Path:**\n\nIn my career, I thrive in roles that allow me to use my organizational skills and attention to detail. I am most effective in structured environments with clear rules and procedures, excelling in fields such as Business Intelligence Analysis, Data Science, or Database Architecture. My strong work ethic and ability to maintain high standards of quality make me an asset in these roles.\n\n### **Relationships:**',
 '4\\. Volunteer / Community Service  \nAt Nakornpayap International School, I proposed having students 

In [ ]:
docs4 = retriever.invoke("What are your hobbies and how did you get into them?")
[doc.page_content for doc in docs4]

['2\\. Extracurricular Activity  \nI was a fingerstyle guitar soloist who performed at our school’s Winter Carnival, Spring Concert, and Class of 2024 Graduation Ceremony. During these events, I played songs such as "Twilight" by Kotaro Oshio, "Like a Star" by Kim Young So, "Always With Me" by Joe Hisaishi, and "Summer" by Joe Hisaishi. This activity spanned from my 11th grade through 12th grade, between December 2022 and June 2024\\.\n\n3\\. Extracurricular Activity  \nAs the leader of our school’s Magic Club during 12th grade, I taught magic techniques and tricks to other students, encouraging them to perform in front of an audience. I believed this would help them explore a unique form of performance art and boost their confidence in social settings. My time commitment for this activity was 2 hours per week for 28 weeks during the 2023-2024 academic year.',
 'Because of my inexhaustible curiosity, I always annoyed my dad with ridiculous questions like this. And ever since then, I li

In [5]:
# Initialize model & tokenizer
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

model_name = "grimjim/mistralai-Mistral-7B-Instruct-v0.3" # ～12m
"""
not working:
    "upstage/solar-pro-preview-instruct"
    "meta-llama/Meta-Llama-3-70B-Instruct"
    "mistralai/Mistral-7B-Instruct-v0.3"
    "mlx-community/Meta-Llama-3-70B-Instruct-4bit" # taking too long
    "tanliboy/lambda-qwen2.5-14b-dpo-test" # taking too long
"""

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
# Initialize LLM
from transformers import pipeline, TextStreamer
from langchain.llms import HuggingFacePipeline

class streamer(TextStreamer):
    def __init__(self, tokenizer, stop_words, **kwargs):
        super().__init__(tokenizer, **kwargs)
        self.stop_words = stop_words
        self.buffer = ""
        self.generated_text = ""
        self.stop = False

    def __call__(self, text_chunk, stream_end=False):
        if self.stop:
            return
        return super().__call__(text_chunk, stream_end)

    def reset(self):
        self.buffer = ""
        self.generated_text = ""
        self.stop = False

    def on_finalized_text(self, text: str, stream_end: bool = False):
        if self.stop:
            return

        self.buffer += text
        # print(self.buffer)

        if any(phrase in text for phrase in self.stop_words) and len(self.buffer) > len(text):
            # print("bef", text, "aft", text.split("</s>")[0])
            new_text = text.split("</s>")[0].split("<|assistant|>")[0].split("<|system|>")[0].split("<|user|>")[0].strip()
            self.generated_text += new_text
            print(new_text, end="", flush=True)
            # print("Stopping generation early.")
            self.stop = True
            return

        if len(self.buffer) > len(text):
            print(text, end="", flush=True)
            # yield(text)

        if stream_end:
            self.buffer = ""
            self.generated_text = ""

# streamer = TextStreamer(tokenizer)
stop_words = ["</s>", "<|user|>", "<|assistant|>", "<|system|>"]
streamer = streamer(tokenizer, stop_words)
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.15,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=600,
    streamer=streamer,
)
"""
worse performance:
    temperature=0.1
    temperature=0.2
    temperature=0.25
    temperature=0.5
    temperature=1

    repetition_penalty=0.1
    repetition_penalty=0.5
    repetition_penalty=1.5
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

<ipython-input-6-198ea28952d1>:74: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [7]:
# Build RAG chain
import re
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# def preprocess_question(inputs):
#     documents = inputs["context"]
#     inputs["context"] = "\n".join(doc.page_content for doc in documents)

#     pronoun_mapping = {"yours": "the individual\'s", "your": "the individual\'s", "you": "the individual"}
#     query = inputs["question"]
#     for key, value in pronoun_mapping.items():
#         query = re.sub(key, value, query)
#     inputs["question"] = query

#     return inputs

embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
"""
worse performance:
    "sentence-transformers/all-MiniLM-L6-v2" (& slower)
"""
def is_relevant(inputs):
    # docs = [doc.page_content for doc in inputs["context"][:3]]
    # context = "\n".join(docs)
    context = inputs["context"][0].page_content
    # print(context)
    question = inputs["question"]

    context_embedding = embedding_model.encode(context)
    question_embedding = embedding_model.encode(question)
    threshold = 0.2

    similarity = cosine_similarity([context_embedding], [question_embedding])[0][0]
    # print(similarity)
    return call_llm if similarity > threshold else "Sorry. I can\'t answer this question."

def postprocess(llm_output):
    response = llm_output.split("<|assistant|>")[1].split("</s>")[0].split("<|system|>")[0].split("<|user|>")[0].strip()
    return response

prompt_template = """
<|system|>

Imagine you are the young university student depicted in the context, answer questions about yourself as if you are \
that individual with first-person pronouns such as "I", "me", and "my" only. Do not use phrases like "the individual" \
or "the person" in your response. Do not mention anything about the documents nor the context given to you. Do not \
include any reasoning processes in the response and make it concise and sound natural. When you can't find an answer \
in the context, or if the question is unrelated to the context, nonsensical, or inappropriate, simply answer "I don't \
know how to answer this question". Use all of the following context to help:
{context}
</s>

<|user|>

{question}
</s>

<|assistant|>

"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

call_llm = prompt | llm | RunnableLambda(postprocess)
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | RunnableLambda(is_relevant) | StrOutputParser()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Answer questions based on given context
def run_rag(question):
    try:
        response = rag_chain.invoke(question)
    except IndexError:
        response = streamer.generated_text
        print("")
    streamer.reset()
    return response

In [ ]:
question0 = "Is Taiwan a country?"
run_rag(question0)

"Sorry. I can't answer this question."

In [ ]:
question1 = "What's your name?"
run_rag(question1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


My name is Leo. Nice to meet you! What can I help you with today?

'My name is Leo. Nice to meet you! What can I help you with today?'

In [ ]:
question2 = "Where are you currently studying at?"
run_rag(question2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I am currently studying at The Hong Kong University of Science and Technology.

'I am currently studying at The Hong Kong University of Science and Technology.'

In [ ]:
question3 = "Could you tell me about yourself and describe your background in brief?"
run_rag(question3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I am a university student who is currently pursuing a Bachelor's degree at the Hong Kong University of Science and Technology (HKUST). My name is Liren Zhang, also known as Leo, and I am 19 years old. I have a strong interest in technology and analytics, which is reflected in my career path. I excel in roles that require organizational skills, attention to detail, and a logical, fact-oriented approach. In my free time, I enjoy watching anime, playing mobile games, reading self-help and technical books, and listening to various types of music. I am also passionate about volunteering and community service, as demonstrated by my proposal to have students serve as classroom interns at Nakornpayap International School. In this role, I worked as an intern in a Calculus BC class, assisting with homework, grading assignments, and other classroom responsibilities.

"I am a university student who is currently pursuing a Bachelor's degree at the Hong Kong University of Science and Technology (HKUST). My name is Liren Zhang, also known as Leo, and I am 19 years old. I have a strong interest in technology and analytics, which is reflected in my career path. I excel in roles that require organizational skills, attention to detail, and a logical, fact-oriented approach. In my free time, I enjoy watching anime, playing mobile games, reading self-help and technical books, and listening to various types of music. I am also passionate about volunteering and community service, as demonstrated by my proposal to have students serve as classroom interns at Nakornpayap International School. In this role, I worked as an intern in a Calculus BC class, assisting with homework, grading assignments, and other classroom responsibilities."

In [ ]:
question4 = "What are your hobbies and how did you get into them?"
run_rag(question4)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I have several hobbies, including reading, playing mobile games, and listening to music. I got into reading by developing an insatiable curiosity about the world around me and wanting to learn more. This led me to start reading books that explained the fundamental concepts behind common things and the underlying workings of machinery and substances. As for mobile games, I found them entertaining and engaging, particularly rhythm games like Cytus II, Deemo, Phigros, and Arcaea. Lastly, I enjoy listening to music from various genres such as C-pop, J-pop, English pop, OSTs, classical, and more. Some of my favorite artists include Jay Chou, JJ Lin, Kenshi Yonezu, Yorushika, YOASOBI, and Imagine Dragons.

'I have several hobbies, including reading, playing mobile games, and listening to music. I got into reading by developing an insatiable curiosity about the world around me and wanting to learn more. This led me to start reading books that explained the fundamental concepts behind common things and the underlying workings of machinery and substances. As for mobile games, I found them entertaining and engaging, particularly rhythm games like Cytus II, Deemo, Phigros, and Arcaea. Lastly, I enjoy listening to music from various genres such as C-pop, J-pop, English pop, OSTs, classical, and more. Some of my favorite artists include Jay Chou, JJ Lin, Kenshi Yonezu, Yorushika, YOASOBI, and Imagine Dragons.'

In [ ]:
question5 = "What subjects are you passionate about?"
run_rag(question5)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I am particularly passionate about Artificial Intelligence (AI) and computer programming. I enjoy exploring the complexities and possibilities of AI algorithms and their applications. Additionally, I have a keen interest in music, specifically playing the guitar, and finding creative ways to connect different songs and perspectives.

'I am particularly passionate about Artificial Intelligence (AI) and computer programming. I enjoy exploring the complexities and possibilities of AI algorithms and their applications. Additionally, I have a keen interest in music, specifically playing the guitar, and finding creative ways to connect different songs and perspectives.'

In [9]:
# Implement streaming
from transformers import TextIteratorStreamer
from threading import Thread
import time

pause_time = 0.145

def stream_response(question, retriever, pipeline):
    inputs = {"context": retriever.invoke(question), "question": question}
    if type(is_relevant(inputs)) == str:
        yield "Sorry. I can\'t answer this question."
        return

    context = "\n".join(doc.page_content for doc in inputs["context"])
    prompt = prompt_template.format(context=context, question=question)
    tokenizer = pipeline.pipeline.tokenizer
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    generation_kwargs = {
        "input_ids": tokenizer(prompt, return_tensors="pt").input_ids.to("cuda"),
        "streamer": streamer,
        "max_new_tokens": 600,
    }
    thread = Thread(target=pipeline.pipeline.model.generate, kwargs=generation_kwargs)
    thread.start()

    buffer = ""
    for text in streamer:
        if any(phrase in text for phrase in stop_words):
            new_text = text.split("</s>")[0].split("<|assistant|>")[0].split("<|system|>")[0].split("<|user|>")[0].strip()
            buffer += new_text
            time.sleep(pause_time)
            yield buffer
            break
        buffer += text
        time.sleep(pause_time)
        yield buffer
    thread.join()

In [10]:
# Gradio function
import gradio as gr

def invoke_rag(question):
    # if not question: raise Exception("Question field empty")
    if not question: return "Feel free to ask any questions about me!"

    # response = run_rag(question)
    # return response
    for text in stream_response(question, retriever, llm):
        yield text

# callback1 = gr.CSVLogger()
# callback2 = gr.CSVLogger()
# callback3 = gr.CSVLogger()

# def receive_feedback1(question, response):
#     callback1.flag([question, response])
#     return "Thank you for your feedback!"

# def receive_feedback2(question, response):
#     callback2.flag([question, response])
#     return "Thank you for your feedback!"

# def receive_feedback3(question, response):
#     callback3.flag([question, response])
#     return "Thank you for your feedback!"

In [11]:
# ans = invoke_rag(question0)
# list(ans)

In [13]:
# Build demo
with gr.Blocks() as demo:
    gr.HTML("<center><h1>Leonidas Bot</h1></center>")

    question = gr.Textbox(label="Question", placeholder="What would you like to know about me?", lines=1)
    response = gr.Textbox(label="Response", lines=10)
    gr.Examples(
        examples=["What's your name?", "Where are you currently studying at?", "Could you tell me about yourself and describe your background in brief?"],
        inputs=question,
        label="Example Questions"
    )
    submit = gr.Button("Ask")
    # with gr.Row():
    #     feedback1 = gr.Button("Answer was useful")
    #     feedback2 = gr.Button("Answer was inaccurate")
    #     feedback3 = gr.Button("Answer was irrelevant")

    # callback1.setup([question, response], "useful")
    # callback2.setup([question, response], "inaccurate")
    # callback3.setup([question, response], "irrelevant")

    gr.on(
        triggers=[question.submit, submit.click],
        fn=invoke_rag,
        inputs=question,
        outputs=response
    )
    # feedback1.click(receive_feedback1, [question, response], [response], preprocess=False)
    # feedback2.click(receive_feedback2, [question, response], [response], preprocess=False)
    # feedback3.click(receive_feedback3, [question, response], [response], preprocess=False)

# demo.launch(share=True, debug=True)
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b9842a9628cb466bae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
demo.close()

Closing server running on port: 7861


In [ ]:
### Feedback
# online name: Leonidas
# favorite color, food, anime, etc.
# update age
# add information about friends
# refuse to answer unrelated and nonsensical and inappropriate questions
# implement streaming